# 03: Enriquecimiento Biográfico

**Propósito:** Este *notebook* enriquece la lista de diputados (`diputados.csv`) con datos biográficos detallados, creando `diputados_bio.csv`.

**Proceso:**
1.  Carga la lista de períodos (`periodos_master.csv`).
2.  Carga la tabla de consulta de BCN (`bcn_url_master_lookup.csv`).
3.  Para cada período:
    a. Carga `diputados.csv`.
    b. Resuelve las URLs de BCN usando *fuzzy matching* contra la tabla de consulta.
    c. Scrapea el texto de las biografías y el distrito desde las URLs encontradas.
    d. Usa un LLM (Ollama) para extraer datos estructurados (JSON) del texto.
    e. Guarda el `DataFrame` enriquecido como `diputados_bio.csv`.

**Dependencias:**
* `data/01_raw/periodos_master.csv` (de `00_Extraction_Periods`)
* `data/01_raw/bcn_url_master_lookup.csv` (de `02_Extraction_BCN_Lookup`)
* `data/01_raw/[periodo]/diputados.csv` (de `01_Extraction_Deputies`)

**Artefactos de Salida:**
* `data/01_raw/[periodo]/diputados_bio.csv`

In [2]:
import pandas as pd
from pathlib import Path
import sys
import logging
import json
from tqdm.notebook import tqdm # Para barras de progreso

# --- Configurar Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Importar lógica personalizada de /src ---
sys.path.append('../') 
try:
    from src.common_utils import sanitize_filename, get_ollama_client
    from src.bio_utils import (
        find_best_match_bcn,    
        scrape_bcn_bio_data,       
        extract_bio_data_llm
    )
except ImportError as e:
    logging.error(f"ERROR: No se pudieron importar las funciones desde /src. {e}")
    logging.error("Asegúrese de que 'src/extraction_utils.py' y 'src/bio_utils.py' existan y contengan las funciones.")
    raise

tqdm.pandas()

In [3]:
# --- 1. Configuración de Rutas y Constantes ---
ROOT = Path.cwd().parent
DATA_DIR_RAW = ROOT / "data" / "01_raw"

# --- Definir ARCHIVOS DE ENTRADA (Dependencias) ---
MASTER_PERIOD_FILE = DATA_DIR_RAW / "periodos_master.csv"
BCN_LOOKUP_FILE = DATA_DIR_RAW / "bcn_url_master_lookup.csv"

# --- Configuración del LLM ---
OLLAMA_HOST = 'http://127.0.0.1:11434'
OLLAMA_MODEL = 'llama3:instruct' # Modelo recomendado
FUZZY_MATCH_THRESHOLD = 70       # Umbral de confianza para el matching

logging.info(f"Ruta Raíz: {ROOT}")
logging.info(f"Directorio de Datos Raw: {DATA_DIR_RAW}")
logging.info(f"Modelo LLM: {OLLAMA_MODEL}")

2025-10-26 11:25:20,618 - INFO - Ruta Raíz: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-
2025-10-26 11:25:20,620 - INFO - Directorio de Datos Raw: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw
2025-10-26 11:25:20,621 - INFO - Modelo LLM: llama3:instruct


## 2. Carga de Dependencias (Períodos, BCN Lookup, Cliente LLM)

Cargamos los artefactos maestros necesarios para el bucle y conectamos con Ollama.

In [7]:
try:
    df_periodos = pd.read_csv(MASTER_PERIOD_FILE)
    logging.info(f"Se cargó la lista maestra de {len(df_periodos)} períodos.")
    
    df_bcn_lookup = pd.read_csv(BCN_LOOKUP_FILE)
    logging.info(f"Se cargó la tabla de consulta BCN con {len(df_bcn_lookup)} registros.")
    
    # Crear la lista de períodos válidos para la función de scraping
    # (Corregido el bug de la variable global 'periodos')
    lista_periodos_validos = df_periodos['Nombre'].tolist()
    logging.info(f"Lista de {len(lista_periodos_validos)} períodos válidos creada.")

except FileNotFoundError as e:
    logging.error(f"ERROR FATAL: No se encontró el archivo de dependencia: {e.filename}")
    logging.error("Por favor, ejecute los notebooks '00' y '02' primero.")
    raise

# Inicializar cliente Ollama (UNA SOLA VEZ)
ollama_client = get_ollama_client(OLLAMA_HOST)
if ollama_client is None:
    logging.warning("No se pudo conectar a Ollama. La extracción de datos estructurados (LLM) se omitirá.")

2025-10-25 22:45:49,901 - INFO - Se cargó la lista maestra de 10 períodos.
2025-10-25 22:45:49,906 - INFO - Se cargó la tabla de consulta BCN con 623 registros.
2025-10-25 22:45:49,907 - INFO - Lista de 10 períodos válidos creada.
2025-10-25 22:45:50,195 - INFO - HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"
2025-10-25 22:45:50,198 - INFO - Cliente Ollama conectado exitosamente en http://127.0.0.1:11434


In [9]:
logging.info(f"Iniciando enriquecimiento biográfico para {len(df_periodos)} períodos...")

choices_bcn = df_bcn_lookup['nombre_en_lista'].tolist()
url_map_bcn = df_bcn_lookup.set_index('nombre_en_lista')['url_wiki'].to_dict()

for row in tqdm(df_periodos.itertuples(), total=len(df_periodos), desc="Procesando Períodos"):
    nombre_periodo = row.Nombre
    nombre_carpeta = sanitize_filename(nombre_periodo)
    carpeta_periodo = DATA_DIR_RAW / nombre_carpeta
    ruta_input_diputados = carpeta_periodo / "diputados.csv"
    ruta_output_bio = carpeta_periodo / "diputados_bio.csv"

    logging.info(f"--- Procesando Período: {nombre_periodo} ---")

    # --- Chequeos ---
    if not ruta_input_diputados.exists():
        logging.warning(f"Saltando período: No se encontró archivo de diputados en {ruta_input_diputados}")
        continue
    if ruta_output_bio.exists():
        logging.info(f"Saltando período: El archivo 'diputados_bio.csv' ya existe.")
        continue

    try:
        # --- 1. Cargar Datos y Crear 'nombre_completo' ---
        df_diputados = pd.read_csv(ruta_input_diputados)
        logging.info(f"Cargados {len(df_diputados)} diputados.")

        try:
            cols_nombre = ['Diputado.Nombre', 'Diputado.ApellidoPaterno', 'Diputado.ApellidoMaterno']
            df_diputados['nombre_completo'] = (
                df_diputados[cols_nombre[0]].astype(str) + ' ' + 
                df_diputados[cols_nombre[1]].astype(str) + ' ' + 
                df_diputados[cols_nombre[2]].astype(str)
            )
            df_diputados['nombre_completo'] = df_diputados['nombre_completo'].str.replace('nan', '', case=False).str.strip().str.replace(r'\s+', ' ', regex=True)
            logging.info("Columna 'nombre_completo' creada.")
        except KeyError as e:
            logging.error(f"ERROR: Faltan columnas de nombre clave {e}. No se puede hacer fuzzy matching.")
            continue 

        # --- 2. OPTIMIZACIÓN DE MATCHING ---
        logging.info("Optimizando: Creando lista de nombres únicos...")
        
        # Obtenemos los nombres únicos del DataFrame de diputados
        nombres_unicos = df_diputados['nombre_completo'].unique()
        logging.info(f"Se encontraron {len(nombres_unicos)} nombres únicos para {len(df_diputados)} filas.")
        
        df_queries = pd.DataFrame(nombres_unicos, columns=['nombre_completo'])

        tqdm.pandas(desc="Fuzzy Matching (Únicos)")
        match_results = df_queries['nombre_completo'].progress_apply(
            lambda nombre_query: find_best_match_bcn(
                nombre_query, 
                choices_bcn, 
                threshold=FUZZY_MATCH_THRESHOLD
            )
        )
        
        df_match = pd.DataFrame(
            match_results.tolist(), 
            index=df_queries.index, 
            columns=['match_nombre_bcn', 'match_score']
        )
        
        df_lookup_final = df_queries.join(df_match)
        df_lookup_final['url_wiki'] = df_lookup_final['match_nombre_bcn'].map(url_map_bcn)

        # --- 3. MAPEO DE VUELTA ---
        df_dip_matched = (pd.merge(
            df_diputados,
            df_lookup_final,
            on='nombre_completo',
            how='left' # Mantiene todos los diputados originales
        ).drop_duplicates(subset=['nombre_completo']))
        # --- 4. Scraping ---
        logging.info("Iniciando scraping de biografías BCN...")
        scraped_data_list = df_dip_matched['url_wiki'].progress_apply(
            lambda url: scrape_bcn_bio_data(url, lista_periodos_validos)
        )
        df_scraped = pd.DataFrame(scraped_data_list.tolist(), index=df_dip_matched.index)
        df_dip_scraped = df_dip_matched.join(df_scraped)
        display(df_dip_scraped.head(3))
        # --- 5. Extracción LLM ---
        logging.info("Iniciando extracción con LLM (Ollama)...")
        if ollama_client:
            bio_data_list = df_dip_scraped['bio_texto_completo'].progress_apply(
                lambda x: extract_bio_data_llm(ollama_client, x, OLLAMA_MODEL)
            )
            df_bio_extraida = pd.DataFrame(
                [data if isinstance(data, dict) else {} for data in bio_data_list],
                index=df_dip_scraped.index
            )
            df_final_enriquecido = pd.concat([df_dip_scraped, df_bio_extraida], axis=1)
        else:
            logging.warning("Omitiendo extracción LLM (cliente no disponible).")
            df_final_enriquecido = df_dip_scraped

        # --- 6. Guardar ---
        df_final_enriquecido.to_csv(ruta_output_bio, index=False, encoding="utf-8")
        logging.info(f"Guardado exitosamente: {ruta_output_bio}")

    except Exception as e:
        logging.error(f"ERROR FATAL al procesar período {nombre_periodo}: {e}", exc_info=True)

logging.info("--- Enriquecimiento biográfico finalizado ---")

2025-10-25 22:46:16,514 - INFO - Iniciando enriquecimiento biográfico para 10 períodos...


Procesando Períodos:   0%|          | 0/10 [00:00<?, ?it/s]

2025-10-25 22:46:16,527 - INFO - --- Procesando Período: 1965-1969 ---
2025-10-25 22:46:16,528 - WARNING - Saltando período: No se encontró archivo de diputados en C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw\1965-1969\diputados.csv
2025-10-25 22:46:16,529 - INFO - --- Procesando Período: 1990-1994 ---
2025-10-25 22:46:16,530 - WARNING - Saltando período: No se encontró archivo de diputados en C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw\1990-1994\diputados.csv
2025-10-25 22:46:16,531 - INFO - --- Procesando Período: 1994-1998 ---
2025-10-25 22:46:16,532 - WARNING - Saltando período: No se encontró archivo de diputados en C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw\1994-1998\diputados.csv
2025-10-25 22:46:16,532 - INFO - --- Procesando Período: 1998-2002 ---
2025-10-25 22:46:16,533 - 

Fuzzy Matching (Únicos):   0%|          | 0/121 [00:00<?, ?it/s]

2025-10-25 22:46:17,042 - INFO - Iniciando scraping de biografías BCN...


...

2025-10-25 23:53:33,613 - INFO - Iniciando extracción con LLM (Ollama)...


Fuzzy Matching (Únicos):   0%|          | 0/163 [00:00<?, ?it/s]

2025-10-25 23:54:19,840 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-25 23:55:16,932 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-25 23:56:11,019 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-25 23:56:37,413 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-25 23:57:03,079 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-25 23:57:25,201 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-25 23:57:44,677 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-25 23:58:25,309 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-25 23:58:57,718 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-25 23:59:30,307 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/cha

2025-10-26 00:36:51,952 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-26 00:37:24,363 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-26 00:37:47,157 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-26 00:38:08,376 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-26 00:38:35,941 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-26 00:39:03,597 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-26 00:39:28,121 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-26 00:39:52,811 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-26 00:40:17,730 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-26 00:41:03,253 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/cha

2025-10-26 01:18:09,725 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-26 01:18:09,743 - INFO - Guardado exitosamente: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw\2018-2022\diputados_bio.csv
2025-10-26 01:18:09,744 - INFO - --- Procesando Período: 2022-2026 ---
2025-10-26 01:18:09,746 - INFO - Saltando período: El archivo 'diputados_bio.csv' ya existe.
2025-10-26 01:18:09,748 - INFO - --- Enriquecimiento biográfico finalizado ---
